# REINFORCE in TensorFlow

Just like we did before for q-learning, this time we'll design a neural network to learn `CartPole-v0` via policy gradient (REINFORCE).

In [1]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it

import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [2]:
import gym
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0")

# gym compatibility: unwrap TimeLimit
if hasattr(env,'env'):
    env=env.env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

#plt.imshow(env.render("rgb_array"))

# Building the policy network

For REINFORCE algorithm, we'll need a model that predicts action probabilities given states.

For numerical stability, please __do not include the softmax layer into your network architecture__. 

We'll use softmax or log-softmax where appropriate.

In [3]:
import tensorflow as tf

# create input variables. We only need <s,a,R> for REINFORCE
states = tf.placeholder('float32', (None,)+state_dim, name="states")
actions = tf.placeholder('int32', name="action_ids")
cumulative_rewards = tf.placeholder('float32', name="cumulative_returns")

In [6]:
import keras

n_hidden_0 = 20
n_hidden_1 = 20

# <define network graph using raw tf or any deep learning library>
model = keras.models.Sequential()
model.add( keras.layers.Dense(n_hidden_0, activation='relu', input_shape=state_dim ) )
model.add( keras.layers.Dense(n_hidden_1, activation='relu') )
model.add( keras.layers.Dense(n_actions, activation='linear' ) )

# logits = <linear outputs (symbolic) of your network>
logits = model(states)

policy = tf.nn.softmax(logits)
log_policy = tf.nn.log_softmax(logits)

In [7]:
# utility function to pick action in one given state
def get_action_proba(s): 
    return policy.eval({states: [s]})[0]

#### Loss function and updates

We now need to define objective and update over policy gradient.

Our objective function is

$$ J \approx  { 1 \over N } \sum  _{s_i,a_i} \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$


Following the REINFORCE algorithm, we can define our objective as follows: 

$$ \hat J \approx { 1 \over N } \sum  _{s_i,a_i} log \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$

When you compute gradient of that function over network weights $ \theta $, it will become exactly the policy gradient.


In [8]:
# select log-probabilities for chosen actions, log pi(a_i|s_i)
indices = tf.stack([tf.range(tf.shape(log_policy)[0]), actions], axis=-1)
log_policy_for_actions = tf.gather_nd(log_policy, indices)

In [9]:
# REINFORCE objective function
# hint: you need to use log_policy_for_actions to get log probabilities for actions taken

# J =  # <policy objective as in the last formula. Please use mean, not sum.>
J = tf.reduce_mean( log_policy_for_actions * cumulative_rewards )

In [10]:
# regularize with entropy
# entropy = <compute entropy. Don't forget the sign!>
entropy = tf.reduce_sum( -policy * tf.log(policy), axis=1 )
entropy = tf.reduce_mean(entropy)

In [11]:
# all network weights
# all_weights = <a list of all trainable weights in your network>
all_weights = model.weights

# weight updates. maximizing J is same as minimizing -J. Adding negative entropy.
loss = -J - 0.1*entropy

update = tf.train.AdamOptimizer().minimize(loss, var_list=all_weights)

### Computing cumulative rewards

In [12]:
def get_cumulative_rewards(rewards,    # rewards at each step
                           gamma=0.99  # discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...

    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently

    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """

    # Calculate the discount factor. Assumes the first rewards are earliest    
    cum_rewards = [np.nan] * len(rewards)
    cum_rewards[-1] = rewards[-1]
    for t in range(1, len(rewards)):
        cum_rewards[-(t+1)] = rewards[-(t+1)] + gamma * cum_rewards[-t] 

    return cum_rewards

In [13]:

assert len(get_cumulative_rewards(np.arange(100))) == 100
assert np.allclose(get_cumulative_rewards([0, 0, 1, 0, 0, 1, 0], gamma=0.9),
                   [1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0, 0, 1, -2, 3, -4, 0], gamma=0.5),
                   [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0, 0, 1, 2, 3, 4, 0], gamma=0),
                   [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


In [14]:
def train_step(_states, _actions, _rewards):
    """given full session, trains agent with policy gradient"""
    _cumulative_rewards = get_cumulative_rewards(_rewards)
    update.run({states: _states, actions: _actions,
                cumulative_rewards: _cumulative_rewards})


### Playing the game

In [15]:
def generate_session(t_max=1000):
    """play env with REINFORCE agent and train at the session end"""

    # arrays to record session
    S, A, R = [], [], []

    s = env.reset()

    for t in range(t_max):

        # action probabilities array aka pi(a|s)
        action_probas = get_action_proba(s)

        a = np.random.choice(n_actions, p=action_probas)

        new_s, r, done, info = env.step(a)

        # record session history to train later
        S.append(s)
        A.append(a)
        R.append(r)
        
        s = new_s
        if done:
            break

    train_step(S, A, R)

    # technical: return session rewards to print them later
    return sum(R)

In [22]:
%%time
sess = tf.InteractiveSession()
init = sess.run(tf.global_variables_initializer())

for i in range(20):

    rewards = [generate_session() for _ in range(100)]  # generate new sessions

    print("mean reward:%.3f" % (np.mean(rewards)))

    if np.mean(rewards) > 500:
        print("You Win!") # but you can train even further
        break

mean reward:22.800
mean reward:25.850
mean reward:31.730
mean reward:44.890
mean reward:55.370
mean reward:79.990
mean reward:133.540
mean reward:211.620
mean reward:224.490
mean reward:207.960
mean reward:238.640
mean reward:278.020
mean reward:393.290
mean reward:236.060
mean reward:112.920
mean reward:207.410
mean reward:297.050
mean reward:186.890
mean reward:237.600
mean reward:217.360
CPU times: user 2min 25s, sys: 10.1 s, total: 2min 35s
Wall time: 2min 19s


### Results & video

In [ ]:
# record sessions
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"),
                           directory="videos", force=True)
sessions = [generate_session() for _ in range(100)]
env.close()

In [ ]:
# show video
from IPython.display import HTML
import os

video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1]))  # this may or may not be _last_ video. Try other indices

In [ ]:
from submit import submit_cartpole
submit_cartpole(generate_session, "cmm801@gmail.com", "kMIBgvAbwVoxkzrj")

In [ ]:
# That's all, thank you for your attention!
# Not having enough? There's an actor-critic waiting for you in the honor section.
# But make sure you've seen the videos first.